<a href="https://colab.research.google.com/github/huongtn1112/BTL_ML_Re-Id/blob/main/Re_Id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 9809 (delta 213), reused 301 (delta 206), pack-reused 9487
Receiving objects: 100% (9809/9809), 9.57 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (7221/7221), done.
/content/deep-person-reid
     |████████████████████████████████| 5.8 MB 5.4 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 185 kB 52.1 MB/s 
     |████████████████████████████████| 42 kB 977 kB/s 
     |████████████████████████████████| 42 kB 843 kB/s 
     |████████████████████████████████| 69 kB 7.6 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.1
    Uninstalling importlib-metadata-4.8.1:
      Successfully uninstalled importlib-metadata-4.8.1
/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython ev

# Import torchreid

In [2]:
import torchreid

# Load data manager

In [3]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3417 KB/s, 43 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




# Build model, optimizer and lr_scheduler

In [4]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# Build engine

In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)


# Run training and test

In [6]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)


epoch: [33/60][100/404]	time 0.448 (0.452)	data 0.001 (0.006)	eta 1:24:28	loss 1.0860 (1.0909)	acc 100.0000 (99.9375)	lr 0.000030
epoch: [33/60][110/404]	time 0.449 (0.452)	data 0.001 (0.006)	eta 1:24:18	loss 1.0735 (1.0901)	acc 100.0000 (99.9432)	lr 0.000030
epoch: [33/60][120/404]	time 0.448 (0.451)	data 0.001 (0.005)	eta 1:24:08	loss 1.1148 (1.0907)	acc 100.0000 (99.9219)	lr 0.000030
epoch: [33/60][130/404]	time 0.447 (0.451)	data 0.001 (0.005)	eta 1:24:00	loss 1.1008 (1.0911)	acc 100.0000 (99.9279)	lr 0.000030
epoch: [33/60][140/404]	time 0.447 (0.450)	data 0.001 (0.005)	eta 1:23:52	loss 1.0690 (1.0914)	acc 100.0000 (99.9330)	lr 0.000030
epoch: [33/60][150/404]	time 0.447 (0.450)	data 0.000 (0.005)	eta 1:23:44	loss 1.0964 (1.0911)	acc 100.0000 (99.9375)	lr 0.000030
epoch: [33/60][160/404]	time 0.451 (0.450)	data 0.001 (0.004)	eta 1:23:37	loss 1.1012 (1.0913)	acc 100.0000 (99.9414)	lr 0.000030
epoch: [33/60][170/404]	time 0.447 (0.450)	data 0.000 (0.004)	eta 1:23:30	loss 1.0925 (1.0